**readme**: <br>
scripts below are used to sample unlabeled data (5000) and tokenized text; <br>
output tokenized unlabeled data is stored under res/tokenized_sampled_unlabeled_reviews

In [ ]:
import pandas as pd
import os
os.chdir('/Users/liyuan/desktop/CSAir/codes')
import re
import csv
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import jieba
import jieba.posseg as pseg
import jieba.analyse
import glob
import codecs
import json

from tokenization import Tokenization
from unlabeled_preprocess import unlabeled_preprocess

In [15]:
pu = unlabeled_preprocess()
fk_reviews = pu.load_unlabeled_data()
# randomly sampled 5000 unlabeled user reviews

反馈数据 in total has 24233 reviews
反馈数据举3例： ['今天的CZ3539航班的毛毯质量很差，下来发现裤子上掉落很多毛。', '最近经常出现提前很久打电话预留前排靠过道位置，到机场就没有，一直道歉，作为高端客户，感觉很不好。', '最近航班服务经常出问题，提前很久打电话预留前排靠过道位置，到机场就没有物流，作为高端客户，对这样的服务很不满意。']


In [ ]:
sampled_unlabeled_reviews = pu.sample_unlabeled(5000,'../res/sampled_unlabeled_reviews.txt')

In [3]:
print('see the first 10 unlabeled reviews ...')
sampled_unlabeled_reviews[:10]

see the first 10 unlabeled reviews ...


['查不到航班',
 '我要取消在线选座的登机牌',
 '本人被额外多办一张会员卡，南航会员办理流程是否需本人办理，而且我在被办理会员卡为郭忠能的时间段乘用过南航飞机，是否为了完成任务利用客户信息多次办理会员卡，对此给本人带来不便，请南航内部严查，避免给客户带来不便，造成客户信息泄漏',
 '打不開。半個小時依然打不開。浪費時間',
 'n/a',
 'app上怎么也查不到还要飞多少里程或航段才能升级？',
 '我在广州白云机场办了南航的积分卡，现在暂时不准备去广东工作，卡不用寄到所写联系地址了，还有工作单位，麻烦把个人资料销毁，谢谢',
 '别人的信息怎么通知到我这里',
 '3月26日搭乘CZ8426从大阪飞深圳，遗留一把伞在飞机上，座位是35K，知道有没有看到，谢谢',
 '2018年05月17日已经反馈.....每次签到都要输入手机号，然后短信回复验证码才能签到，这是不是跟手机营业厅串通赚取短信费？.....还没看到回复？？？']

In [4]:
# re-process unlabeled data, including raw untokenized data
sents_tok_lst = []
stopwords = [line.strip() for line in open('../Source_Data/stopwords.txt', 'r', encoding='utf-8').readlines()]   
sents = sampled_unlabeled_reviews
for sent in sents:
    tokenized_sent = ' '.join(word for word in jieba.cut(sent.strip()) if word not in stopwords)
    # remove digit
    tokenized_sent = re.sub(r'\d+','',tokenized_sent)
    # remove float
    tokenized_sent = re.sub(r"(\d+\.\d+)", '', tokenized_sent)
    tokenized_sent = re.sub(r'[^\w\s]','', tokenized_sent)
    # remove non-chinese characters
    re_words = re.compile(u"[\u4e00-\u9fa5]+")
    res = re.findall(re_words, tokenized_sent)
    if res:
        tokenized_sent = ' '.join([r for r in res])
    sents_tok_lst.append(tokenized_sent)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/_d/b3chzbkx5vgg1wtjm942qj4m0000gn/T/jieba.cache
Loading model cost 3.051 seconds.
Prefix dict has been built succesfully.


In [9]:
print('Total length of tokenized unlabeled user review %d'%len(sents_tok_lst))
print('take a look at the first 10 tokenized unlabeled data ...')
sents_tok_lst[:10]

Total length of tokenized unlabeled user review 5000
take a look at the first 10 tokenized unlabeled data ...


['查不到 航班',
 '我要 取消 在线 选座 登机牌',
 '额外 多办 一张 会员卡 南航 会员 办理 流程 需 办理 办理 会员卡 郭忠能 时间段 乘用 南航 飞机 利用 客户 信息 办理 会员卡 对此 带来 不便 请 南航 内部 严查 客户 带来 不便 客户 信息 泄漏',
 '打不開 半個 小時 依然 打不開 浪費 時間',
 'n a',
 '查不到 飞 里程 航段 升级',
 '广州白云机场 办 南航 积分 卡 暂时 广东 工作 卡 不用 寄到 写 联系地址 工作 单位 麻烦 个人资料 销毁 谢谢',
 '信息 通知',
 '月 日 搭乘 大阪 飞 深圳 遗留 一把 伞 飞机 座位 有没有 谢谢',
 '年 月 日 反馈 每次 签到 输入 手机号 短信 回复 验证码 签到 手机 营业厅 串通 赚取 短信 费 回复']

In [14]:
# format unlabeled into a df
data = {}
data['reviews'] = sampled_unlabeled_reviews
data['review_tokens'] = sents_tok_lst
unlabeled_df = pd.DataFrame(data)
unlabeled_df.head()

# output to csv file
unlabeled_df.to_csv('../res/unlabeled_review_5000.csv', index = False)

,reviews,review_tokens
0,查不到航班,查不到 航班
1,我要取消在线选座的登机牌,我要 取消 在线 选座 登机牌
2,本人被额外多办一张会员卡，南航会员办理流程是否需本人办理，而且我在被办理会员卡为郭忠能的时间...,额外 多办 一张 会员卡 南航 会员 办理 流程 需 办理 办理 会员卡 郭忠能 时间段 乘...
3,打不開。半個小時依然打不開。浪費時間,打不開 半個 小時 依然 打不開 浪費 時間
4,n/a,n a
